In [84]:
# Open the file
from collections import defaultdict, deque, Counter, namedtuple
from functools import reduce, cache, cmp_to_key
from heapq import heappush, heappop, heapify
from itertools import permutations, combinations, product
from math import gcd, sqrt, factorial, ceil, floor
from sortedcontainers import SortedDict, SortedList, SortedSet
from pyrival import * # python competitive programming library
from typing import TypeAlias
from math import prod
import re
import numpy as np
import sys
import pyomo.environ as pymo
import pyomo.opt as pyopt
sys.setrecursionlimit(10000)
def scan_nums(line):
    # Use a regular expression to find numbers
    numbers = re.findall(r'-?\d+\.?\d*', line)
    # Convert to appropriate numerical types
    numbers = [float(num) if '.' in num else int(num) for num in numbers]
    return numbers

def get_blocks(input):
    blocks = []
    cur_block = []
    for line in input:
        if line == '\n':
            blocks.append(cur_block)
            cur_block = []
        else:
            cur_block.append((line.strip()))    
    blocks.append(cur_block)
    return blocks
        
input_file = 'input.txt'
test_file = 'test.txt'

with open(input_file, 'r') as file:
    lines  = file.readlines()
    blocks = get_blocks(lines)
    # get grid 
    raw_grid = blocks[0]
  
    grid = [list(row) for row in raw_grid]
    moves = [c for line in blocks[-1] for c in line]
    
# where is the robot 

def get_robot(grid):
    R = len(grid)
    C = len(grid[0])
    for r in range(R):
        for c in range(C):
            if grid[r][c] == '@':
                return r, c
    print('no robot')

r,c = get_robot(grid)



def inbounds(r,c, grid):
    R = len(grid)
    C = len(grid[0])
    return  0 <= r < R and 0 <= c < C
# now we need to simulate this
# try slow solu for part 1 
# when push a box, push whole line of boxes until they hit wall 

# dictionary of what direction each signal implies 
move_dir = {'^':(-1,0), 'v':(1,0), '<':(0,-1), '>':(0,1)}

def evolve_one(grid, r, c, move):
    """Update the board for one move of the robot"""
    dr, dc = move_dir[move]
    nr, nc = r + dr, c + dc 
    if not inbounds(nr, nc, grid) or grid[nr][nc] == '#':
        return r, c
    if grid[nr][nc] == '.':
        grid[r][c] = '.'
        r = nr
        c = nc
        grid[nr][nc] = '@'
        return r, c
    else: 
        rs, cs = r + dr, c + dc 
        while inbounds(rs, cs, grid) and grid[rs][cs] == 'O':
            rs += dr
            cs += dc 
        if not inbounds(rs, cs, grid) or grid[rs][cs] == '#':
            return r, c
        
        else:
            grid[r][c] = '.'
            grid[nr][nc] = '@'
            grid[rs][cs] = 'O'
            r, c = nr, nc
            
            return r, c
        
def display(grid):
    """Output the grid in a clean way"""
    for row in grid:
        print(''.join(row))
    print('/n')
        
def evolve(grid, moves):
    """ updates the board for the whole moveset """
    r, c = get_robot(grid)
    for move in moves:
        r, c  = evolve_one(grid, r, c, move)


def score_grid(grid):
    """ scores the resulting grid """
    R = len(grid)
    C = len(grid[0])
    score = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == 'O':
                score += 100 *(r + 1) + (c + 1)
    return score

evolve(grid, moves)
print(score_grid(grid))
            


                
                
    
                
    
        

1408579


In [33]:
with open(test_file, 'r') as file:
    lines  = file.readlines()
    blocks = get_blocks(lines)
    # get grid 
    raw_grid = blocks[0]
    R = len(raw_grid) - 2
    C = len(raw_grid) - 2
    grid = [list(row) for row in raw_grid]

        
def transformed(grid):
    R = len(grid)
    C = len(grid[0])
    new_grid = []
    for row in grid:
        new_row = []
        for c in row:
            if c == '.':
                new_row.append('.')
                new_row.append('.')
            elif c == 'O':
                new_row.append('[')
                new_row.append(']')
                
            elif c == '@':
                new_row.append('@')
                new_row.append('.')
            else:
                new_row.append('#')
                new_row.append('#')
        new_grid.append(new_row)
    return new_grid


new_grid = transformed(grid)
display(new_grid)                
        

['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']
['#', '.', '.', 'O', '.', '.', 'O', '.', 'O', '#']
['#', '.', '.', '.', '.', '.', '.', 'O', '.', '#']
['#', '.', 'O', 'O', '.', '.', 'O', '.', 'O', '#']
['#', '.', '.', 'O', '@', '.', '.', 'O', '.', '#']
['#', 'O', '#', '.', '.', 'O', '.', '.', '.', '#']
['#', 'O', '.', '.', 'O', '.', '.', 'O', '.', '#']
['#', '.', 'O', 'O', '.', 'O', '.', 'O', 'O', '#']
['#', '.', '.', '.', '.', 'O', '.', '.', '.', '#']
['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################
/n


In [85]:
# now a challenge is that it can push tons of boxes in weird ways and need to know if some part of the "group" of boxes is up against an obstacle.

# pushing horizontally is the same. 
# but pushing 

with open(input_file, 'r') as file:
    lines  = file.readlines()
    blocks = get_blocks(lines)
    # get grid 
    raw_grid = blocks[0]
    R = len(raw_grid) - 2
    C = len(raw_grid) - 2
    grid = [list(row) for row in raw_grid]
   
    moves = [c for line in blocks[-1] for c in line]

# helpful for swapping in horizontal case
opp_box = {'[': ']', ']': '['}

def evolve_horizontal(grid, r, c, move):
    """ Updates the grid given robot at (r,c) with the specified move"""
    dr, dc = move_dir[move]
    nr, nc = r + dr, c + dc 
    # Case 1: cant move, against boundary
    if not inbounds(nr, nc, grid) or grid[nr][nc] == '#':
        return r, c
    # Case 2: spot is open 
    if grid[nr][nc] == '.':
        grid[r][c] = '.'
        r = nr
        c = nc
        grid[nr][nc] = '@'
        return r, c
    # Case 3: Hitting a box 
    else: 
        # get how long the row of boxes goes 
        rs, cs = r + dr, c + dc 
        while inbounds(rs, cs, grid) and grid[rs][cs] == ']' or grid[rs][cs] == '[':
            rs +=  dr
            cs +=  dc 
        # goes to edge, can't push
        if not inbounds(rs, cs, grid) or grid[rs][cs] == '#':
            return r, c
        # can push! 
        else:
            # update grid 
            grid[r][c] = '.'
            grid[nr][nc] = '@'
            grid[rs][cs] = grid[rs - dr][cs - dc]
            # push inermediate
            for i in range(1, abs(cs - nc)):
                new_chr = opp_box[grid[nr + i * dr][nc + i * dc]]
                grid[nr + i * dr][nc + i * dc] = new_chr
            # update pos of robot 
            r, c = nr, nc
            return r , c

def can_move_vert(grid, r, c, move):
    """returns true if the box with left position at (r, c)"""
    dr, dc = move_dir[move]
    
    nr, nc = r + dr, c + dc 
    if not inbounds(nr, nc, grid) or grid[nr][nc] == '#' or grid[nr][nc + 1] == '#':
        return False
    elif grid[nr][nc] == '.' and grid[nr][nc + 1] == '.':
        return True
    else:
        if grid[nr][nc] == '[':
            return can_move_vert(grid, nr, nc, move)
        
        left = can_move_vert(grid, nr, nc - 1, move) if grid[nr][nc] == ']' else True
        right = can_move_vert(grid, nr, nc + 1, move) if grid[nr][nc + 1] == '[' else True
        return left and right 

def move_boxes_vert(grid, r, c, move):
    """will move the boxes given we can. r, c is left side of box """
    dr, dc = move_dir[move]
    nr, nc = r + dr, c + dc
    
    if grid[nr][nc] == '.' and grid[nr][nc + 1] == '.' :
        grid[nr][nc] = grid[r][c]
        grid[nr][nc + 1] = grid[r][c + 1]

        grid[r][c] = '.'
        grid[r][c + 1] = '.'
        return
    else:
        # move other boxes first
        if grid[nr][nc] == '[':
            move_boxes_vert(grid, nr, nc, move)
            
        if grid[nr][nc] == ']':
            move_boxes_vert(grid, nr, nc - 1, move)
            
        if grid[nr][nc + 1] == '[':
            move_boxes_vert(grid, nr, nc + 1, move)
        # move current box
        grid[nr][nc] = grid[r][c]
        grid[nr][nc + 1] = grid[r][c + 1]
        grid[r][c] = '.'
        grid[r][c + 1] = '.'
        return
            
            
def evolve_vert(grid, r, c, move):
    """ evolves the grid on a vertical turn and gives new robot position"""
    dr, dc = move_dir[move]
    nr, nc = r + dr, c + dc 
    # can't move 
    if not inbounds(nr, nc, grid) or grid[nr][nc] == '#':
        return r, c
    # empty spot to move to
    if grid[nr][nc] == '.':
        grid[r][c] = '.'
        r = nr
        c = nc
        grid[nr][nc] = '@'
        return r, c
    else: 
        # we are hitting a box, does it move if we push?
        rs, cs = r + dr, c + dc 
        moveable = can_move_vert(grid, rs, cs, move) if grid[rs][cs] == '[' else can_move_vert(grid, rs, cs - 1, move)
        # Yes? push it and the ones it pushes 
        if moveable:
            rs, cs = (rs, cs) if grid[rs][cs] == '[' else (rs, cs - 1)
            move_boxes_vert(grid, rs, cs, move)
            grid[r][c] = '.'
            r = nr
            c = nc
            grid[nr][nc] = '@'
            return r, c
        # cant move 
        return r, c
                
            
    
    
def evolve_warehouse(grid, moves):
    """ evolves the thiccer warehouse over the set of moves"""
    r, c = get_robot(grid)
    for move in moves:
        
        if move in ['<', '>']:
            r, c = evolve_horizontal(grid, r, c, move)
        else:
            r, c = evolve_vert(grid, r, c, move)

def score_warehouse(grid):
    """ score the thicc warehouse"""
    R = len(grid)
    C = len(grid[0])
    score = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == '[':
                vert_dist = r 
                hor_dist = c  
                score += 100 * (vert_dist) + hor_dist
    return score

evolve_warehouse(new_grid, moves)
print(score_warehouse(new_grid))
display(new_grid)

            

1379173
####################################################################################################
##..[]...[][][][][]........[]...##....[][][]..[][][][][][]##[]####[]##..[][]....[]......[]..[][]..##
####[]..[][]####[]......[]......[]##.[][].[]##[][][][]....[][][][][].[][].....[][]....[]##......[]##
##..[].[][][].............[]......[][][][]...[]............................[][][][]...##..##....[]##
##..[]##[]..............##..##.....[]......[]...[]..............##......[][]##[].....[]...........##
##[].[][]...............##....[]...[]...[]...................[].........[].[].....................##
##....##....................[][][]...[]...........[]..##..............##..##[][]##[]..............##
####....[]......##...[]...................##.......[].............[].[]...[][]##[]................##
##.................[]................................[]...........##[][]....[][][][][][]..[][]....##
##[]..............[]##.................[].................##......[]...[].....##[][